In [1]:
import numpy as np
import pandas as pd

In [2]:
d = {'Clubes': [], 'PG': [], 'j': [], 'V': [], 'E': [], 'D': [], 'GP': [], 'GC': [], 'SG': []}
dfClassChampionship = pd.DataFrame(data=d);
dfDataChampionship = pd.read_excel("b2010.xlsx", encoding='utf8')

In [3]:
dfDataChampionship.head()

,Horario,Dia,Data,Clube 1,Clube 2,Placar,Vencedor,Rodada,Arena
0,18h30,Sábado,08/05/2010,Botafogo,Santos,3x3,-,1ª Rodada,Engenhão
1,18h30,Sábado,08/05/2010,Atlético-GO,Grêmio,0x0,-,1ª Rodada,Serra Dourada
2,18h30,Sábado,08/05/2010,Palmeiras,Vitória,1x0,Palmeiras,1ª Rodada,Parque Antártica
3,16h00,Domingo,09/05/2010,Flamengo,São Paulo,1x1,-,1ª Rodada,Maracanã
4,16h00,Domingo,09/05/2010,Atlético-MG,Vasco,2x1,Atlético-MG,1ª Rodada,Mineirão


** Get all clubes for championship **



In [4]:
#Cancatenando as colunas Clube 1 e Clube 2
clubs  =  pd.concat([dfDataChampionship['Clube 1'].str.lower(), dfDataChampionship['Clube 2'].str.lower()], axis=1, keys=['Clubes'])

In [5]:
#Obtem todos os clubes 
cb = pd.DataFrame({'Clubes': clubs['Clubes'].unique()})

In [6]:
#Todos os clubes que participaram dp campeonato
cb

,Clubes
0,botafogo
1,atlético-go
2,palmeiras
3,flamengo
4,atlético-mg
5,internacional
6,corinthians
7,ceará
8,guarani
9,avaí


In [7]:
def getNumberWinner(data, clube ):
    df = data.apply(lambda x: x.str.strip())
    filter = df["Vencedor"].str.lower() == clube
    return (data[filter]['Vencedor'].count()).astype(np.int64)

def getNumberDepartures(data, clube ):
    df = data.apply(lambda x: x.str.strip())
    filter1 = df["Clube 1"].str.lower() == clube
    filter2 = df["Clube 2"].str.lower() == clube
    return (data[filter1]['Clube 1'].count() + data[filter2]['Clube 2'].count()).astype(np.int64)

def getPoints(data, clube ):
    df = data.apply(lambda x: x.str.strip())
    filter1 = df["Clube 1"].str.lower() == clube
    filter2 = df["Clube 2"].str.lower() == clube
    filter3 = df["Vencedor"].str.lower() == clube
    filter4 = (df["Clube 1"].str.lower() == clube) | (df["Clube 2"].str.lower() == clube)
    filter5 = df["Vencedor"].str.lower() == '-'
    
    v1 = data[(filter1) & (filter3)]
    v1 = v1['Vencedor'].count()
    v2 = data[(filter2) & (filter3)]
    v2 = v2['Vencedor'].count() 
    v3 = data[(filter4) & (filter5)]
    v3 = v3['Vencedor'].count()
    return ((v2*3)+(v1*3)+v3).astype(np.int64)

def getDraw(data, clube ):
    df = data.apply(lambda x: x.str.strip())
    filter1 = (df["Clube 1"].str.lower() == clube) | (df["Clube 2"].str.lower() == clube)
    filter2 = df["Vencedor"].str.lower() == '-'
    df = data[(filter1) & (filter2)]
    empates = df['Vencedor'].count()
    return empates.astype(np.int64)

def getDefeats(data, clube ):
    df = data.apply(lambda x: x.str.strip())
    filter1 = (df["Clube 1"].str.lower() == clube) | (df["Clube 2"].str.lower() == clube)
    filter2 = (df["Vencedor"].str.lower() != clube) & (df["Vencedor"].str.lower() != '-')
    df = data[(filter1) & (filter2)]
    derrotas = df['Vencedor'].count()
    return derrotas.astype(np.int64)

def getGP(data, clube ):
    df = data.apply(lambda x: x.str.strip())
    filter1  =  df["Clube 1"].str.lower() == clube
    filter2  =  df["Clube 2"].str.lower() == clube
    df1      =  data[(filter1)]
    df2      =  data[(filter2)]
    placar1  =  df1['Placar'].str.split('x')
    placar2  =  df2['Placar'].str.split('x')
    gp       =  0
    gc       =  0
    for g1, g2 in placar1:
        gp = (gp + pd.to_numeric( g1 ))
        gc = (gc + pd.to_numeric( g2 ))
            
    for g1, g2 in placar2:
        gp = (gp + pd.to_numeric( g2 )) 
        gc = (gc + pd.to_numeric( g1 ))
    return gp, gc

In [14]:
dfTable = cb[["Clubes"]].copy()
for column in ["PG", "J", "V", "E", "D", "GP", "GC", "SG"]:
    dfTable[column] = pd.np.nan
    
    
for index, row in dfTable.iterrows():
    c1  =  row['Clubes']
    if type(c1) != unicode:
        c1 =  c1.decode('utf-8')
    c1 =  c1.strip()
    
    pg       =  getPoints(dfDataChampionship, c1 )
    j        =  getNumberDepartures(dfDataChampionship, c1 )
    v        =  getNumberWinner(dfDataChampionship, c1 )
    e        =  getDraw(dfDataChampionship, c1 )
    d        =  getDefeats(dfDataChampionship, c1 )
    gp, gc   =  getGP(dfDataChampionship, c1 )
    
    if c1 == u'grêmio prudente':
        pg -= 3
    
    dfTable.at[index, 'PG'] = pg
    dfTable.at[index, 'J']  = j
    dfTable.at[index, 'V']  = v
    dfTable.at[index, 'E']  = e
    dfTable.at[index, 'D']  = d
    dfTable.at[index, 'GP'] = gp
    dfTable.at[index, 'GC'] = gc
    dfTable.at[index, 'SG'] = gp - gc
    
    #print c1

dfTable

,Clubes,PG,J,V,E,D,GP,GC,SG
0,botafogo,59.0,38.0,14.0,17.0,7.0,54.0,42.0,12.0
1,atlético-go,42.0,38.0,11.0,9.0,18.0,51.0,57.0,-6.0
2,palmeiras,50.0,38.0,12.0,14.0,12.0,42.0,43.0,-1.0
3,flamengo,44.0,38.0,9.0,17.0,12.0,41.0,44.0,-3.0
4,atlético-mg,45.0,38.0,13.0,6.0,19.0,52.0,64.0,-12.0
5,internacional,58.0,38.0,16.0,10.0,12.0,48.0,41.0,7.0
6,corinthians,68.0,38.0,19.0,11.0,8.0,65.0,41.0,24.0
7,ceará,47.0,38.0,10.0,17.0,11.0,35.0,44.0,-9.0
8,guarani,37.0,38.0,8.0,13.0,17.0,33.0,53.0,-20.0
9,avaí,43.0,38.0,11.0,10.0,17.0,49.0,58.0,-9.0


In [15]:
reorderlist = [ 'Fluminense', 'Cruzeiro', 'Corinthians', 'Grêmio', 'Atlético-pr', 'Botafogo', 'Internacional',
                'Santos', 'São Paulo', 'Palmeiras', 'Vasco', 'Ceará', 'Atlético-mg', 'Flamengo', 'Avaí',
                'Atlético-go', 'Vitória', 'Guarani', 'Goiás', 'Grêmio Prudente']
listIndex   = []

#Processa a lista para gerar a lista de ordenação
for clube in reorderlist:
    for index, row in dfTable.iterrows():
        #Verifica se o valor de clube é um unicode, se não, convete isso.
        if type(clube) != unicode:
            c =  clube.decode('utf-8')
        else:
            c =  clube
        
        c1  =  row['Clubes']
        #print c1
        if type(c1) != unicode:
            c1 =  c1.decode('utf-8')
        
        c1 =  c1.strip()
        c  =  c.lower()
        
        #Verifica se o clube iterado é igual ao clube da lista, se sim
        #retorna seu indice
        if c1 == c:
            listIndex.append( index )
            break
            
#Reordena a lista
dfTable  =  dfTable.reindex(index=listIndex, columns=["Clubes", "PG", "J", "V", "E", "D", "GP", "GC", "SG"])
#Com a lista ordenada, corrige os indices.
dfTable.index =  np.arange(1, len(dfTable)+1)
dfTable

,Clubes,PG,J,V,E,D,GP,GC,SG
1,fluminense,71.0,38.0,20.0,11.0,7.0,62.0,36.0,26.0
2,cruzeiro,69.0,38.0,20.0,9.0,9.0,53.0,38.0,15.0
3,corinthians,68.0,38.0,19.0,11.0,8.0,65.0,41.0,24.0
4,grêmio,63.0,38.0,17.0,12.0,9.0,68.0,43.0,25.0
5,atlético-pr,60.0,38.0,17.0,9.0,12.0,43.0,45.0,-2.0
6,botafogo,59.0,38.0,14.0,17.0,7.0,54.0,42.0,12.0
7,internacional,58.0,38.0,16.0,10.0,12.0,48.0,41.0,7.0
8,santos,56.0,38.0,15.0,11.0,12.0,63.0,50.0,13.0
9,são paulo,55.0,38.0,15.0,10.0,13.0,54.0,54.0,0.0
10,palmeiras,50.0,38.0,12.0,14.0,12.0,42.0,43.0,-1.0


** O Grêmio Prudente foi punido com a perda de 3 pontos pela escalação **
** irregular do zagueiro Paulão, na partida contra o Flamengo, em 23/05. **

In [ ]:
** Por algum motivo que desconheço, na fonte dos dados o Atlético Paranaense tem GC 44 ao invés de 45, estão o SG está re **